In [1]:
#Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
#Load to files
schools_csv_path = "Resources/schools_complete.csv"
students_csv_path = "Resources/students_complete.csv"

In [3]:
#Read the files and store into dataframes
schools_df = pd.read_csv(schools_csv_path)
students_df = pd.read_csv(students_csv_path)

In [4]:
#delete
schools_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [5]:
#delete
students_df.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
dtype: object

In [6]:
#combine the data into a single dataset
#want to use the school ID as the index... 
combined_df = pd.merge(students_df, schools_df, on="school_name")

In [7]:
#delete
combined_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [8]:
#delete - testing for missing values
combined_df.count()

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
School ID        39170
type             39170
size             39170
budget           39170
dtype: int64

In [9]:
#a passing math scores is 70 or greater, a passing reading score is 70 or greater, and overall is if they pass both
passing_math = combined_df['math_score'] > 70
passing_reading = combined_df['reading_score'] > 70

In [10]:
passing_math

0         True
1        False
2        False
3        False
4         True
         ...  
39165     True
39166    False
39167     True
39168     True
39169     True
Name: math_score, Length: 39170, dtype: bool

In [11]:
passing_reading

0        False
1         True
2         True
3        False
4         True
         ...  
39165     True
39166     True
39167     True
39168     True
39169     True
Name: reading_score, Length: 39170, dtype: bool

In [12]:
#passing_overall #is when both passing reading AND math... not sure how to do it

In [13]:
#count number of true for reading and math

In [15]:
# Calculate the total number of backers for all US projects (from kickstarter activity)
total_students = students_df['student_name'].count()
total_students

39170

In [16]:
average_math_score = students_df["math_score"].mean()
average_math_score

78.98537145774827

In [17]:
average_read_score = students_df["reading_score"].mean()
average_read_score

81.87784018381414

In [18]:
# Calculate the average number of backers for all US projects (from kickstarter activity)
total_budget = schools_df["budget"].sum()
total_budget

24649428

In [20]:
total_schools = schools_df["school_name"].count()
total_schools

15

In [ ]:
#add passing math to a new df
#summary_combined_df = [['passing_math','passing_reading']]

In [ ]:
#create a (data frame) summary with Total Schools, Total Students, Total Budget, Average Math Score, Average Reading Score, %Passing Math, %Passing Reading %Overall Passing
summary_schools_df = pd.DataFrame({"Total Schools": [total_schools], 
                                   "Total Students": total_students, 
                                   "Total Budget": total_budget, 
                                   "Average Math Score": average_math_score,
                                   "Average Reading Score": average_read_score})
                                   #"% Passing Math": pass_math,"% Passing Reading": pass_read, "% Overall Passing": overall_pass})
summary_schools_df

In [ ]:
#Create a dataframe with metrics about each school it should groupby school and include the following columns: (school name as index), School Type, Total Students, Total School Budget, Per Student Budget, Average Math Score, Average Reading Score, % Passing Math, % Passing Reading 
#to sort from highest to lowest ascending=False must be passed in
#example new_name_df = name_of_df.sort_values("Column_name",ascending=False)

In [ ]:
#sort the schools by descending order and display the top five performing schools by overall passing

#sorted_df = 


In [ ]:
#sort the schools by ascending order and display the top five bottom performing schools by overall passing


In [ ]:
#create a data frame that lists the average reading score for students of each grade level (use bins) grade levels are 9th,10th, 11th, and 12th -- index by school

#create a pandas series for each grade (use a conditional statement)

#combine series into a dataframe


In [ ]:
#create a data frame that lists the average math score for students of each grade level (use bins) grade levels are 9th,10th, 11th, and 12th -- index by school

#create a pandas series for each grade (use a conditional statement)

#combine series into a dataframe


In [ ]:
#create a dataframe that breaks down school performance based on average spending ranges (per student) -- use four reasonable bins to group schools by spending.

#a table should include the following: Spending Ranges (Per Student), Average Math Score, Average Raading Score, % Passing Math, %Passing Reading, %Overall Passing


In [ ]:
#create a dataframe that breaks down school performance based on school size -- use four reasonable bins to group schools.

#a table should include the following: Spending Ranges (Per Student), Average Math Score, Average Raading Score, % Passing Math, %Passing Reading, %Overall Passing


In [ ]:
#create a dataframe that breaks down school performance based on school type (charter or district)

#a table should include the following: Spending Ranges (Per Student), Average Math Score, Average Raading Score, % Passing Math, %Passing Reading, %Overall Passing
